### "If after drudges you make the code work, just remember that CPU is a rock you tricked into thinking - Unknown!"

### Status of Task: Completed All three tasks!!! 
### I will demonstrate three methods to tackle with such kind of problems
####  Method1: Spark-nlp  by John Snow Labs
#### Why Spark-nlp for this solution?
#### Since we are dealing with text data (on spark),  its not any harm to assume the end consumer of the scrapped data could be some Natural Language Processing (NLP) pipline. NLP actually tockenises, vectorises, and do compute/memory intensive mathmatical operations such as word2vec on textual data under the hood. So that is the reason why I choose spark-nlp to demonstrat first part of the Task. 

#### Method 2: Regular Transformations, Actions Operations on RDDs
#### I tried to solve Second part of Task using regular RDD Operations, they are self explined and I have added comments

#### Method 3: Using custom Beautiful Soup user defined functions (UDF) in Spark. [The most versitle and easy]

##### Most of the data is on RDDs except Pandas DF.  I transformed RDD dfs to Pandas dfs before writing to csv. One can still directly write from RDDs using RDD joins or SQL queries on RDDs, but my time ran out so I had to do Pandas which is quick and easy. But anyway when one calls Action task the data is dumped into the driver node instantly. My code just adds Pandas layer before dumping to driver node. One can even use Pandas API built on top of Spark.  That way it will be lazy loading and final dump will take place on the final Action opertion. 

In [2]:
#prepare environment:
#One can use conda environment:
#cd /path/to/directory/where/files/are/located
#conda create -n pyspark_env
#conda activate pyspark_env
#conda install -c conda-forge pyspark

#Export path to where pyspark is installed:
#export SPARK_HOME=/path/to/pyspark #Check correct path! its important otherwise spark will not run
#optional if you want to launch from other terminals too
#source ~/.bashrc

#try if spark runs:
#spark-shell #works

#Install spark-nlp #https://nlp.johnsnowlabs.com/docs/en/install
#conda install -c johnsnowlabs spark-nlp

#Export python path for both driver and worker nodes:
#python version shuld be same for both
#thats why it is important to work in conda/Python virtual environment,so as to not to clash the python versions
# in both nodes.
#export PYSPARK_PYTHON="path/to/python" # path shoudl be something like cwd/conda/env/bin/...
#export PYSPARK_DRIVER_PYTHON="path/to/python"

#install Beautiful Soup too
#conda install -c anaconda beautifulsoup4

In [3]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SQLContext, Row
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from pyspark.ml import Pipeline
#from py4j.java_gateway import java_import
import sparknlp
sc = sparknlp.start()
#sc.stop()

/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
sqlContext = SQLContext(sc)

/home/yogender/.local/lib/python3.6/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [5]:
#sc = SparkContext()
#sqlContext = SQLContext(sc)

In [6]:
rdd_l = sc.sparkContext.wholeTextFiles("./*.html")

In [7]:
df = rdd_l.toDF(schema=["filename", "text"]).select("text")

In [8]:
rules = '''

.\d\&\w+\;\d+&\w+;\d+&\w+;Kč* 

'''
#\S*\d+\S* Kč* 
with open('regex_rules.txt', 'w') as f:
    
    f.write(rules)

In [9]:
RegexMatcher().extractParamMap()

{Param(parent='RegexMatcher_5afd0e9697c6', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='RegexMatcher_5afd0e9697c6', name='strategy', doc='MATCH_FIRST|MATCH_ALL|MATCH_COMPLETE'): 'MATCH_ALL'}

In [10]:




documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("assembled")

regex_matcher = RegexMatcher()\
    .setInputCols('assembled')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./regex_rules.txt', delimiter=',')

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    regex_matcher
 ])

pipelineModel = nlpPipeline.fit(df)
#df = pipelineModel.transform(df)
#empty_df = spark.createDataFrame([['']]).toDF("text")

#pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(df)

match_df.select('regex_matches.result').show(truncate=50)

+------------------------------+
|                        result|
+------------------------------+
|[21&nbsp;500&nbsp;000&nbsp;Kč]|
|[>5&nbsp;200&nbsp;000&nbsp;Kč]|
|[10&nbsp;854&nbsp;900&nbsp;Kč]|
|[>5&nbsp;565&nbsp;000&nbsp;Kč]|
|[>1&nbsp;475&nbsp;000&nbsp;Kč]|
|[>5&nbsp;500&nbsp;000&nbsp;Kč]|
|[>2&nbsp;530&nbsp;000&nbsp;Kč]|
|[>7&nbsp;511&nbsp;000&nbsp;Kč]|
|[>1&nbsp;850&nbsp;000&nbsp;Kč]|
|[>7&nbsp;490&nbsp;000&nbsp;Kč]|
+------------------------------+



In [11]:
#Convert df to rdd and flatten the list of list
match_df_flat = match_df.select("regex_matches.result").rdd.flatMap(lambda x: x[0])

In [12]:
#collect on driver node
match_df_flat.collect()

['21&nbsp;500&nbsp;000&nbsp;Kč',
 '>5&nbsp;200&nbsp;000&nbsp;Kč',
 '10&nbsp;854&nbsp;900&nbsp;Kč',
 '>5&nbsp;565&nbsp;000&nbsp;Kč',
 '>1&nbsp;475&nbsp;000&nbsp;Kč',
 '>5&nbsp;500&nbsp;000&nbsp;Kč',
 '>2&nbsp;530&nbsp;000&nbsp;Kč',
 '>7&nbsp;511&nbsp;000&nbsp;Kč',
 '>1&nbsp;850&nbsp;000&nbsp;Kč',
 '>7&nbsp;490&nbsp;000&nbsp;Kč']

In [106]:
# Do Some cleaning to make it more human readable form. &nbsp is some kind of HTML strings.
#HTML strings are not problems with beautiful soup. Soup automatically parses them and cleans them out.
#But this is an spark-nlp application so I am just demonstrating how to clean unwanted strings from spark rdd
#Using intrinsic map transformation.
match_df_flat1 = match_df_flat.map(lambda s: s.replace('&nbsp;', '')) \
.map(lambda s: s.replace('>', '')) \
.map(lambda s: s.replace('>', '')) \
.map(lambda s: s.replace('Kč', ' Kč')) \
.map(lambda s: s.split()) \
.map(lambda s: '{:,} {}'.format(int(s[0]), str(s[1])))
#.map(lambda s: int(s[0]))
#.map(lambda s: {} {}.format(int(s[0]), str(Kč)))
#.map(lambda s: int(s[0]) + str('Kč'))


In [107]:
match_df_flat1.collect()

['21,500,000 Kč',
 '5,200,000 Kč',
 '10,854,900 Kč',
 '5,565,000 Kč',
 '1,475,000 Kč',
 '5,500,000 Kč',
 '2,530,000 Kč',
 '7,511,000 Kč',
 '1,850,000 Kč',
 '7,490,000 Kč']

In [108]:
match_df_flat1.map(lambda s: s.split())

PythonRDD[103] at RDD at PythonRDD.scala:53

In [109]:
match_df_flat1.collect()

['21,500,000 Kč',
 '5,200,000 Kč',
 '10,854,900 Kč',
 '5,565,000 Kč',
 '1,475,000 Kč',
 '5,500,000 Kč',
 '2,530,000 Kč',
 '7,511,000 Kč',
 '1,850,000 Kč',
 '7,490,000 Kč']

In [110]:
#match_df = match_df.rdd.flatMap(lambda xs: [x[0] for x in xs])

In [111]:
#match_df .collect(

In [112]:
#name the rdd 'cost' we will use it to index kind of
from pyspark.sql import Row

row = Row("cost") 
match_df_flat1_df= match_df_flat1.map(row).toDF()


In [113]:
match_df_flat1_df

DataFrame[cost: string]

In [114]:
match_df_pd_cost = match_df_flat1_df.toPandas()


In [115]:
match_df_pd_cost

,cost
0,"21,500,000 Kč"
1,"5,200,000 Kč"
2,"10,854,900 Kč"
3,"5,565,000 Kč"
4,"1,475,000 Kč"
5,"5,500,000 Kč"
6,"2,530,000 Kč"
7,"7,511,000 Kč"
8,"1,850,000 Kč"
9,"7,490,000 Kč"


In [116]:
#Some house keeping
#rdd_l.map(lambda x: x[1]) takes the data out of tuple of rdd. because rdd when read using WholeTextFile stores
# tuple(fileNamle, StringData).
#.map(lambda x: x.split('\n\t')). i split the data by combination of next line and tab seperater.
#.map(lambda x: x[6]) Pick up the 6th row from rdds as it is the intended class="location-text ng-binding"
#.map(lambda x: x.split('prodeji')), split by prodej to get meaningfull string
#.map(lambda x: x[1]) some indexing to pick elemnt out of list of list
#.map(lambda x: x.split(';')) split it at ; because thats where address seperates 
# .map(lambda x: x[0]) again pick element from lol (list of list) or tuple
# .map(lambda x: x.strip(' ')) strip out empty space
rdd1 = rdd_l.map(lambda x: x[1]).map(lambda x: x.split('\n\t')).map(lambda x: x[6]) \
.map(lambda x: x.split('prodeji')).map(lambda x: x[1]).map(lambda x: x.split(';')) \
.map(lambda x: x[0]).map(lambda x: x.strip(' ')).map(lambda x: str(x))

rows_split_addr = rdd1.map(lambda p: Row(p))

print(rows_split_addr.collect())

schemaString = "address"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
addressSchema = sqlContext.createDataFrame(rows_split_addr, schema)
#print(addressSchema.collect())
addressSchema.repartition(1).write.csv(path='./data.csv', mode="append", header="true")
addressSchema.collect()

[<Row('Masná, Praha 1 - Staré Město')>, <Row('Velvarská, Horoměřice')>, <Row('Libušina, Karlovy Vary')>, <Row('Pod Lipami, Praha 3 - Žižkov')>, <Row('Drnovská, Praha 6 - Ruzyně')>, <Row('V Průhoně, Mukařov - Žernovka')>, <Row('Mladoboleslavská, Mělník')>, <Row('Grafická, Praha 5 - Smíchov')>, <Row('Sametová, Liberec - Liberec VI-Rochlice')>, <Row('Jakubská, Praha 1 - Staré Město')>]


[Row(address='Masná, Praha 1 - Staré Město'),
 Row(address='Velvarská, Horoměřice'),
 Row(address='Libušina, Karlovy Vary'),
 Row(address='Pod Lipami, Praha 3 - Žižkov'),
 Row(address='Drnovská, Praha 6 - Ruzyně'),
 Row(address='V Průhoně, Mukařov - Žernovka'),
 Row(address='Mladoboleslavská, Mělník'),
 Row(address='Grafická, Praha 5 - Smíchov'),
 Row(address='Sametová, Liberec - Liberec VI-Rochlice'),
 Row(address='Jakubská, Praha 1 - Staré Město')]

In [117]:
#Convert RDD dataframe to pandas DF
address_pd_df = addressSchema.toPandas()

In [118]:
address_pd_df

,address
0,"Masná, Praha 1 - Staré Město"
1,"Velvarská, Horoměřice"
2,"Libušina, Karlovy Vary"
3,"Pod Lipami, Praha 3 - Žižkov"
4,"Drnovská, Praha 6 - Ruzyně"
5,"V Průhoně, Mukařov - Žernovka"
6,"Mladoboleslavská, Mělník"
7,"Grafická, Praha 5 - Smíchov"
8,"Sametová, Liberec - Liberec VI-Rochlice"
9,"Jakubská, Praha 1 - Staré Město"


In [119]:
import pandas as pd
df_concat_cost_address = pd.concat([match_df_pd_cost,address_pd_df], axis=1)

In [120]:
df_concat_cost_address

,cost,address
0,"21,500,000 Kč","Masná, Praha 1 - Staré Město"
1,"5,200,000 Kč","Velvarská, Horoměřice"
2,"10,854,900 Kč","Libušina, Karlovy Vary"
3,"5,565,000 Kč","Pod Lipami, Praha 3 - Žižkov"
4,"1,475,000 Kč","Drnovská, Praha 6 - Ruzyně"
5,"5,500,000 Kč","V Průhoně, Mukařov - Žernovka"
6,"2,530,000 Kč","Mladoboleslavská, Mělník"
7,"7,511,000 Kč","Grafická, Praha 5 - Smíchov"
8,"1,850,000 Kč","Sametová, Liberec - Liberec VI-Rochlice"
9,"7,490,000 Kč","Jakubská, Praha 1 - Staré Město"


In [121]:
rdd_l_split = rdd_l.map(lambda x: x[1])

In [122]:
from pyspark.sql import Row

row = Row("val") # Or some other column name
rdd_df = rdd_l_split.map(row).toDF()

In [152]:
import pandas as pd
from bs4 import BeautifulSoup
def processRecord(rdd_l):
    
    #import lxml.html.usedoctest
    #page = urlopen(url)
    #root = lxml.html.fromstring(tag_soup)
    soup = BeautifulSoup(rdd_l, "html.parser")
    #body = fromstring(tag_soup).find('.//body')
    #rtnVal = soup.prettify()
    #print (list(soup.children))
    classes = []

    for element in soup.find_all('ul', class_='params1'):
        for il in element.find_all('li'):
            text = il.get_text(strip=True).replace(u'\xa0', u' ')
            #text2 = il2.get_text(strip=True).replace(u'\xa0', u' ')
            #print ('text1 is :',text1)
            #print ('text2 is :',text2)
            classes.append(text)
            #classes2.append(text2)
    #df = pd.DataFrame([classes1, classes1])
    return classes


In [153]:
convertUDF = lambda z: processRecord(z)

In [154]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [155]:
text_df_cleaned=rdd_df.withColumn('cleaned', udf(convertUDF, StringType())('val'))

In [156]:
text_df_cleaned_flat = text_df_cleaned.select("cleaned").rdd.flatMap(lambda x: x)

In [157]:
from bs4 import BeautifulSoup
from pyspark.sql import Row

row = Row("cleanedParams1") 

text_df_cleaned_flat= text_df_cleaned_flat.map(row).toDF()

In [158]:
pd.set_option('display.max_colwidth', None)

In [159]:
text_df_cleaned_flat_param1_pd = text_df_cleaned_flat.select('cleanedParams1').toPandas()

In [160]:
text_df_cleaned_flat_param1_pd 

,cleanedParams1
0,"[Celková cena:21 500 000 Kč za nemovitost, včetně provize, ID zakázky:32863, Aktualizace:16.02.2021, Stavba:Cihlová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní]"
1,"[Celková cena:5 200 000 Kč za nemovitost, včetně provize, ID zakázky:729030, Aktualizace:08.02.2021, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:2. podlaží z celkem 3, Užitná plocha:78m2, Plocha podlahová:78m2]"
2,"[Celková cena:10 854 900 Kč (420 000 EUR) za nemovitost, včetně provize, Aktualizace:26.01.2021, ID:261724, Stavba:Smíšená, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Podlaží:2. podlaží, Užitná plocha:132m2, Balkón:, Parkování:]"
3,"[Celková cena:5 565 000 Kč za nemovitost, včetně provize, Poznámka k ceně:+ poplatky 3.500 ,- Kč, ID zakázky:N03409, Aktualizace:18.02.2021, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:4. podlaží z celkem 6, Užitná plocha:55m2, Plocha podlahová:55m2]"
4,"[Celková cena:1 475 000 Kč za nemovitost, včetně provize, Poznámka k ceně:včetně právního servisu a provize RK, ID zakázky:0253-NP01146, Aktualizace:11.02.2021, Anuita:4 Kč, Stavba:Cihlová, Stav objektu:Novostavba, Vlastnictví:Družstevní, Převod do OV:Ne, Umístění objektu:Centrum obce, Podlaží:4. podlaží z celkem 6, Užitná plocha:57m2, Plocha podlahová:57m2]"
5,"[Celková cena:5 500 000 Kč za nemovitost, Poznámka k ceně:cena je včetně provize a služeb RK, ID zakázky:NPDO4772, Aktualizace:15.02.2021, Stavba:Cihlová, Stav objektu:Novostavba, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:1. podlaží z celkem 2, Užitná plocha:66m2]"
6,"[Celková cena:2 530 000 Kč za nemovitost, + provize RK, včetně právního servisu (k jednání), Poznámka k ceně:, 3500, ID zakázky:1579/1830, Aktualizace:15.02.2021, Stavba:Panelová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní, Podlaží:1. podlaží z celkem 5, Užitná plocha:44m2, Plocha podlahová:44m2]"
7,"[Celková cena:7 511 000 Kč za nemovitost, + provize RK, včetně právního servisu, Poznámka k ceně:+ provize RK, ID zakázky:202009Graf, Aktualizace:14.02.2021, Stavba:Cihlová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní, Umístění objektu:Centrum obce, Podlaží:4. podlaží z celkem 5 včetně 1 podzemního, Užitná plocha:58m2, Datum nastěhování:Ihned, Rok kolaudace:2021]"
8,"[Celková cena:1 850 000 Kč za nemovitost, Náklady na bydlení:3000,-Kč, Aktualizace:04.02.2021, ID:695644, Stavba:Panelová, Stav objektu:Před rekonstrukcí, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:10. podlaží, Užitná plocha:33m2]"
9,"[Celková cena:7 490 000 Kč za nemovitost, včetně provize, ID zakázky:N05057, Aktualizace:26.10.2020, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Podlaží:3. podlaží z celkem 4 včetně 1 podzemního, Užitná plocha:39m2]"


In [132]:
import pandas as pd
def processRecord2(file):
    
    #import lxml.html.usedoctest
    #page = urlopen(url)
    #root = lxml.html.fromstring(tag_soup)
    soup = BeautifulSoup(file, "html.parser")
    #body = fromstring(tag_soup).find('.//body')
    #rtnVal = soup.prettify()
    #print (list(soup.children))
    classes = []

    for element in soup.find_all('ul', class_='params2'):
        for il in element.find_all('li'):
            text = il.get_text(strip=True).replace(u'\xa0', u' ')
            #text2 = il2.get_text(strip=True).replace(u'\xa0', u' ')
            #print ('text1 is :',text1)
            #print ('text2 is :',text2)
            classes.append(text)
            #classes2.append(text2)
    #df = pd.DataFrame([classes1, classes1])
    return classes

In [133]:
convertUDF2 = lambda z: processRecord2(z)

In [134]:
text_df_cleaned_para2=rdd_df.withColumn('cleaned2', udf(convertUDF2, StringType())('val'))

In [135]:
text_df_cleaned_flat_para2 = text_df_cleaned_para2.select("cleaned2").rdd.flatMap(lambda x: x)

In [136]:
from pyspark.sql import Row

row = Row("cleanedParams2") 

text_df_cleaned_flat_para2= text_df_cleaned_flat_para2.map(row).toDF()

In [137]:
pd.set_option('display.max_colwidth', None)

In [138]:
text_df_cleaned_flat_para2_pd = text_df_cleaned_flat_para2.select('cleanedParams2').toPandas()

In [139]:
text_df_cleaned_flat_para2_pd

,cleanedParams2
0,"[Podlaží:2. podlaží, Užitná plocha:136m2, Plocha podlahová:136m2, Topení:Lokální plynové, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná č. 78/2013 Sb. podle vyhlášky]"
1,"[Parkování:1, Voda:Dálkový vodovod, Topení:Lokální plynové, Odpad:Veřejná kanalizace, Telekomunikace:Internet, Elektřina:230V, Doprava:MHD, Autobus, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná č. 148/2007 Sb. podle vyhlášky, Vybavení:Částečně, Výtah:]"
2,"[Voda:Dálkový vodovod, Odpad:Veřejná kanalizace, Doprava:Silnice, MHD, Autobus, Komunikace:Asfaltová, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná, Bezbariérový:, Vybavení:, Výtah:, Typ bytu:Mezonet]"
3,"[Balkón:2m2, Sklep:2m2, Topení:Ústřední dálkové, Plyn:Plynovod, Telekomunikace:Telefon, Internet, Kabelová televize, Elektřina:230V, Doprava:Silnice, MHD, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná č. 78/2013 Sb. podle vyhlášky, Vybavení:, Výtah:]"
4,"[Balkón:4m2, Sklep:, Garáž:1, Topení:Ústřední dálkové, Odpad:Veřejná kanalizace, Telekomunikace:Kabelová televize, Elektřina:230V, Doprava:MHD, Komunikace:Asfaltová, Energetická náročnost budovy:Třída B - Velmi úsporná č. 78/2013 Sb. podle vyhlášky, Vybavení:, Výtah:]"
5,"[Plocha podlahová:66m2, Parkování:2, Rok kolaudace:2020, Voda:Dálkový vodovod, Odpad:Jímka, Telekomunikace:Internet, Doprava:Silnice, Autobus, Komunikace:Dlážděná, Energetická náročnost budovy:Třída B - Velmi úsporná]"
6,"[Sklep:2m2, Voda:Dálkový vodovod, Topení:Ústřední dálkové, Odpad:Veřejná kanalizace, Telekomunikace:Kabelové rozvody, Elektřina:230V, 400V, Doprava:Vlak, Silnice, MHD, Autobus, Komunikace:Asfaltová, Bezbariérový:]"
7,"[Rok rekonstrukce:2020, Voda:Dálkový vodovod, Topení:Ústřední dálkové, Odpad:Veřejná kanalizace, Telekomunikace:Telefon, Internet, Elektřina:230V, Doprava:Vlak, Dálnice, Silnice, MHD, Autobus, Komunikace:Asfaltová, Energetická náročnost budovy:Třída A - Mimořádně úsporná č. 78/2013 Sb. podle vyhlášky, Bezbariérový:, Výtah:, Typ bytu:Mezonet]"
8,"[Sklep:3m2, Voda:Dálkový vodovod, Topení:Jiné, Odpad:Veřejná kanalizace, Telekomunikace:Telefon, Internet, Kabelová televize, Elektřina:230V, Doprava:MHD, Komunikace:Asfaltová, Výtah:]"
9,"[Plocha podlahová:39m2, Sklep:, Datum nastěhování:Ihned, Voda:Dálkový vodovod, Plyn:Plynovod, Energetická náročnost budovy:Třída D - Méně úsporná č. 78/2013 Sb. podle vyhlášky, Vybavení:, Výtah:]"


In [140]:
df_concat_cost_address_params = pd.concat([df_concat_cost_address,text_df_cleaned_flat_param1_pd,text_df_cleaned_flat_para2_pd ], axis=1)

In [141]:
df_concat_cost_address_params

,cost,address,cleanedParams1,cleanedParams2
0,"21,500,000 Kč","Masná, Praha 1 - Staré Město","[Celková cena:21 500 000 Kč za nemovitost, včetně provize, ID zakázky:32863, Aktualizace:16.02.2021, Stavba:Cihlová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní]","[Podlaží:2. podlaží, Užitná plocha:136m2, Plocha podlahová:136m2, Topení:Lokální plynové, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná č. 78/2013 Sb. podle vyhlášky]"
1,"5,200,000 Kč","Velvarská, Horoměřice","[Celková cena:5 200 000 Kč za nemovitost, včetně provize, ID zakázky:729030, Aktualizace:08.02.2021, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:2. podlaží z celkem 3, Užitná plocha:78m2, Plocha podlahová:78m2]","[Parkování:1, Voda:Dálkový vodovod, Topení:Lokální plynové, Odpad:Veřejná kanalizace, Telekomunikace:Internet, Elektřina:230V, Doprava:MHD, Autobus, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná č. 148/2007 Sb. podle vyhlášky, Vybavení:Částečně, Výtah:]"
2,"10,854,900 Kč","Libušina, Karlovy Vary","[Celková cena:10 854 900 Kč (420 000 EUR) za nemovitost, včetně provize, Aktualizace:26.01.2021, ID:261724, Stavba:Smíšená, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Podlaží:2. podlaží, Užitná plocha:132m2, Balkón:, Parkování:]","[Voda:Dálkový vodovod, Odpad:Veřejná kanalizace, Doprava:Silnice, MHD, Autobus, Komunikace:Asfaltová, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná, Bezbariérový:, Vybavení:, Výtah:, Typ bytu:Mezonet]"
3,"5,565,000 Kč","Pod Lipami, Praha 3 - Žižkov","[Celková cena:5 565 000 Kč za nemovitost, včetně provize, Poznámka k ceně:+ poplatky 3.500 ,- Kč, ID zakázky:N03409, Aktualizace:18.02.2021, Stavba:Cihlová, Stav objektu:Velmi dobrý, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:4. podlaží z celkem 6, Užitná plocha:55m2, Plocha podlahová:55m2]","[Balkón:2m2, Sklep:2m2, Topení:Ústřední dálkové, Plyn:Plynovod, Telekomunikace:Telefon, Internet, Kabelová televize, Elektřina:230V, Doprava:Silnice, MHD, Energetická náročnost budovy:Třída G - Mimořádně nehospodárná č. 78/2013 Sb. podle vyhlášky, Vybavení:, Výtah:]"
4,"1,475,000 Kč","Drnovská, Praha 6 - Ruzyně","[Celková cena:1 475 000 Kč za nemovitost, včetně provize, Poznámka k ceně:včetně právního servisu a provize RK, ID zakázky:0253-NP01146, Aktualizace:11.02.2021, Anuita:4 Kč, Stavba:Cihlová, Stav objektu:Novostavba, Vlastnictví:Družstevní, Převod do OV:Ne, Umístění objektu:Centrum obce, Podlaží:4. podlaží z celkem 6, Užitná plocha:57m2, Plocha podlahová:57m2]","[Balkón:4m2, Sklep:, Garáž:1, Topení:Ústřední dálkové, Odpad:Veřejná kanalizace, Telekomunikace:Kabelová televize, Elektřina:230V, Doprava:MHD, Komunikace:Asfaltová, Energetická náročnost budovy:Třída B - Velmi úsporná č. 78/2013 Sb. podle vyhlášky, Vybavení:, Výtah:]"
5,"5,500,000 Kč","V Průhoně, Mukařov - Žernovka","[Celková cena:5 500 000 Kč za nemovitost, Poznámka k ceně:cena je včetně provize a služeb RK, ID zakázky:NPDO4772, Aktualizace:15.02.2021, Stavba:Cihlová, Stav objektu:Novostavba, Vlastnictví:Osobní, Umístění objektu:Klidná část obce, Podlaží:1. podlaží z celkem 2, Užitná plocha:66m2]","[Plocha podlahová:66m2, Parkování:2, Rok kolaudace:2020, Voda:Dálkový vodovod, Odpad:Jímka, Telekomunikace:Internet, Doprava:Silnice, Autobus, Komunikace:Dlážděná, Energetická náročnost budovy:Třída B - Velmi úsporná]"
6,"2,530,000 Kč","Mladoboleslavská, Mělník","[Celková cena:2 530 000 Kč za nemovitost, + provize RK, včetně právního servisu (k jednání), Poznámka k ceně:, 3500, ID zakázky:1579/1830, Aktualizace:15.02.2021, Stavba:Panelová, Stav objektu:Po rekonstrukci, Vlastnictví:Osobní, Podlaží:1. podlaží z celkem 5, Užitná plocha:44m2, Plocha podlahová:44m2]","[Sklep:2m2, Voda:Dálkový vodovod, Topení:Ústřední dálkové, Odpad:Veřejná kanalizace, Telekomunikace:Kabelové rozvody, Elektřina:230V, 400V, Doprava:Vlak, Silnice, MHD, Autobus, Komunikace:Asfaltová, Bezbariérový:]"
7,"7,511,000 Kč","Grafická, Praha 5 

In [142]:
df_concat_cost_address_params.to_csv("cleaned_data.csv", sep=',')